## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint
from sklearn.datasets import make_moons

In [2]:
# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

In [3]:
model_data.head(3)

,Angaus,SegSumT,SegTSeas,SegLowFlow,DSDist,DSMaxSlope,USAvgT,USRainDays,USSlope,USNative,DSDam,Method,LocSed
0,0,16.0,-0.10,1.036,50.20,0.57,0.09,2.470,9.8,0.81,0,electric,4.8
1,1,18.7,1.51,1.003,132.53,1.15,0.20,1.153,8.3,0.34,0,electric,2.0
2,0,18.3,0.37,1.001,107.44,0.57,0.49,0.847,0.4,0.00,0,spo,1.0


In [4]:
eval_data.head(3)

,Angaus_obs,SegSumT,SegTSeas,SegLowFlow,DSDist,DSMaxSlope,USAvgT,USRainDays,USSlope,USNative,DSDam,Method,LocSed
0,0,16.6,1.01,1.017,5.23,0.29,-1.40,1.980,10.0,1.00,0,electric,4.9
1,1,16.8,-0.51,1.002,2.24,0.00,0.27,0.460,0.7,0.00,0,electric,2.3
2,0,16.3,0.76,1.023,162.28,5.14,-0.60,0.806,21.4,0.66,0,electric,4.3


In [5]:
# generate a moons dataset
X,y = make_moons(n_samples = 1000, noise = 0.2, random_state = 808)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 808)

# Define XGBoost model
model = xgb.XGBClassifier(eval_metric = "logloss", random_state = 808)

# Define hyperparameter dists
param_dist = {}

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [6]:
# Create X, y variables and assign
X = model_data.drop('Angaus', axis=1)
y = model_data['Angaus']

# Encode categorical features
le = LabelEncoder()
X['Method'] = le.fit_transform(X['Method'])

# Split data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=808)

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [7]:
# Create model
xgb_model = XGBClassifier(
    n_estimators=1000,
    learning_rate=0.1,
    eval_metric="logloss",
    random_state=808,
    early_stopping_rounds=50
)

# Fit model with early stopping
xgb_model.fit(
    X_train, y_train,
    eval_set= [(X_val, y_val)],
    verbose = False
)

# Get and print best number of trees
best_ntrees = xgb_model.best_iteration
print(f"Best number of trees: {best_ntrees}")

Best number of trees: 29


### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [8]:
# Create model with best number of trees
model = XGBClassifier(
    n_estimators= best_ntrees,
    eval_metric="logloss",
    random_state=808
)

In [9]:
# Define hyperparameter dist
param_dist = {
    'learning_rate': uniform(0.01, 0.3)
}

In [10]:
# Set up randomized search
random_search = RandomizedSearchCV(
    model,
    param_distributions = param_dist,
    n_iter = 20,
    cv = 5,
    random_state = 808
)

In [11]:
# Fit random search model
random_search.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# Get best learning rate
best_lrate = random_search.best_params_['learning_rate']
print(f"Best learning rate: {best_lrate}")

### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [ ]:
# Model with best trees and learning rate
model = XGBClassifier(
    n_estimators = best_ntrees,
    learning_rate = best_lrate,
    eval_metric = "logloss",
    random_state = 808
)

In [ ]:
# Define hyperparameter dists
param_dist_new = {
    'max_depth': randint(3, 11),
    'min_child_weight': randint(1, 11),
    'gamma': uniform(0.05, 0.05)
}

In [ ]:
# Set up randomized search
random_search_new = RandomizedSearchCV(
    model,
    param_distributions = param_dist_new,
    n_iter = 20,
    cv = 5,
    random_state = 808
)

In [ ]:
# Fit random search
random_search.fit(X_train, y_train)

In [ ]:
# Get best tree parameters
best_tree = random_search_new.best_params_
print(f"Best tree parameters: {best_tree}")

### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [ ]:
# Initialize model with best number of trees, learning rate, and tree parameters
model = XGBClassifier(
    n_estimators=best_n_estimators,
    learning_rate=best_learning_rate,
    **best_tree_params,
    eval_metric="logloss",
    random_state=808
)

In [ ]:
# Define parameter distributions for stochastic parameters
param_dist = {
    'subsample': uniform(0.5, 0.5),  # uniform(0.5, 0.5) gives range 0.5-1.0
    'colsample_bytree': uniform(0.5, 0.5)
}

# Set up randomized search
random_search = RandomizedSearchCV(
    model,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    random_state=808,
    n_jobs=-1
)

# Fit random search
random_search.fit(X_train, y_train)

# Get best stochastic parameters
best_stochastic_params = random_search.best_params_
print(f"Best stochastic parameters: {best_stochastic_params}")

### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

*Your answer here*